<div style="background-image: url('https://cdn-images-1.medium.com/max/1200/1*OOWSoWHeQ5kyJ4N0P2ptNA.png');padding-top: 40px">
<p style="text-align: center;">
  <a href="https://itb.ac.id/" style="margin-top:20px;">
    <img src="https://seeklogo.com/images/I/institut-teknologi-bandung-logo-7B8F816823-seeklogo.com.png" alt="Logo" width=100 height=100>
  </a>
  <div style="text-align: center;font-weight:bold; margin:5px; font-size:30px;font-family: fantasy;">IF3270 Machine Learning</div>
  <p style="text-align: center;margin:0px;font-family: fantasy;font-size:20px">
    Implementasi Algoritma FFNN
    <br style="text-align: center;">
    <br style="text-align: center;">
  </p>
  <p style="text-align: center; margin:0px; margin-top:50px; font-weight: bold; font-size: 30px; font-family: fantasy;">Anggota</p>
  <div style="display:flex; flex-wrap:wrap; text-align:center; flex-direction: row; padding:50px 0;">
    <div style="line-height:1px; flex: 25%; margin:auto;">
      <img src="https://i.ibb.co/HY222Nj/rexy.jpg" width=150 height=150 style="border-radius:50%; margin:auto;object-fit:cover;" >
      <p style="font-family:Courier; font-weight:bold; font-size:20px;text-align:center;">13519010</p>
      <p style="font-family:Courier; font-size:20px;text-align:center;">Rexy</p>
    </div>
    <div style="line-height:1px; flex: 25%; margin:auto;">
      <img src="https://i.ibb.co/xGmzX02/louis.jpg" width=150 height=150 style="border-radius:50%; margin:auto;object-fit:cover;" >
      <p style="font-family:Courier; font-weight:bold; font-size:20px;text-align:center;">13519016</p>
      <p style="font-family:Courier; font-size:20px;text-align:center;">Louis</p>
    </div>
    <div style="line-height:1px; flex: 25%; margin:auto;">
      <img src="https://i.ibb.co/gDxQKbx/dito.jpg" width=150 height=150 style="border-radius:50%; margin:auto;object-fit:cover;" >
      <p style="font-family:Courier; font-weight:bold; font-size:20px;text-align:center;">13519159</p>
      <p style="font-family:Courier; font-size:20px;text-align:center;">Benidictus</p>
    </div>
    <div style="line-height:1px; flex: 25%; margin:auto;">
      <img src="https://i.ibb.co/NmWB3BH/kevin.jpg" width=150 height=150 style="border-radius:50%; margin:auto;object-fit:cover;" >
      <p style="font-family:Courier; font-weight:bold; font-size:20px;text-align:center;">13519216</p>
      <p style="font-family:Courier; font-size:20px;text-align:center;">Kevin</p>
    </div>
  </div>
</p>
</div>

In [1]:
!pip install -r requirements.txt

In [2]:
# import library
import json
import math
import numpy as np

In [3]:
# read model file from filesystem
f = open("model2-sigmoid.json", "r")

# loads model and parse to json object
model = json.loads(f.read())

network_depth = model["Network_Depth"]
learning_rate = model["Learning_Rate"]
num_of_input = model["Num_Of_Input"]
layer_group = model["Layers"]
weight_group = model["Weights"]
input_group = model["Input"]

In [4]:
class Layer:
    def __init__(self, width, activation=None):
        '''
        [Attributes]
            _width      : int       { banyak node pada layer }
            _activation : func      { fungsi aktivasi layer, None jika layer input }
            nodes       : list[float] / list[int] { nilai setiap node pada layer }
        '''
        self._width = width
        self._type = activation
       
        # Set the activation function self._activation = None
        self._activation = self.linear
        if activation is not None:
            if activation == "linear":
                self._activation = self.linear
            if activation == "sigmoid":
                self._activation = self.sigmoid
            if activation == "relu":
                self._activation = self.relu
            if activation == "softmax":
                self._activation = self.softmax
            
        self.nodes = [0] * self._width
    
    def set_nodes(self, val=0):
        '''
        [DESC]
            Menginisialisasi nilai setiap node pada layer
        [PARAMS]
            val     : int/float         { nilai yang akan diinisialisasi untuk setiap node pada layer }
            val     : list[int]|list[float]
        '''
        if isinstance(val, (int, float)):
            self.nodes = [val] * self._width
        if isinstance(val, list):
            self.nodes = val
    
    '''Activation Functions'''
    def activate(self):
        '''
        [DESC]
            Abstraksi yang dipanggil di NeuralNetwork untuk melakukan
            fungsi aktivasi pada setiap node di layer ini
        '''
        self._activation()

    def sigmoid(self):
        '''
        [DESC]
            Mengubah attr nodes dari perkalian vektor weight dengan node
            menjadi list dengan rentang nilai 0 - 1
        [PARAMS]
            
        '''
        for i, z in enumerate(self.nodes):
            self.nodes[i] = 1 / (1 + math.exp(-z))
    
    def linear(self):
        '''
        [DESC]
            Mengubah attr nodes dari perkalian vektor weight dengan node
        [PARAMS]
            X   : list[float]     { input node }
            W   : list[float]     { weight setiap node }
        '''
        pass

    def relu(self):
        '''
        [DESC]
            Menghitung hasil fungsi aktivasi relu.
            Jika nilai dibawah 0 maka kembalian adalah 0, Jika tidak maka mengembalikan nilai dari fungsi linear.
        [PARAMS]
            X   : nilai dari masukan dalam bentuk array of array (Matrix)
            W   : weight dalam bentuk array of array (Matrix)
        '''
        for i, z in enumerate(self.nodes):
            self.nodes[i] = max(0, z)
    
    def softmax(self):
        '''
        [DESC]
            Mengubah attr nodes dengan nilai perbandingan seluruh node
        '''
        temp_z_exps = [np.exp(zi) for zi in self.nodes] # calculate exp(z) for all node in layer
        self.sum_exp_z = np.sum(temp_z_exps)
        for i, z in enumerate(temp_z_exps):
            self.nodes[i] = z / self.sum_exp_z

In [5]:
class NeuralNetwork:
    def __init__(self, data):
        '''
        [ATTRIBUTES]
            n_
        Nilai data diisi dari model
        '''
        self.depth = data["Network_Depth"]
        self.n_input = data["Num_Of_Input"]
        
        self._layers = []
        for i, layer in enumerate(data["Layers"]):
            assert i == layer["depth"]  # check that the json file did not miss a layer
            n_node = layer["width"]
            activation = layer["activation"]
            temp_layer = Layer(width=n_node, activation=activation)
            self._layers.append(temp_layer)
        
        self._weights = [0] * len(data["Weights"])
        for i, weight in enumerate(data["Weights"]):
            assert i == weight["depth_origin"]
            self._weights[i] = weight["values"]
            

    def init_input_layer(self, input_instance):
        '''
        [DESC]
            Menginisialisasi input layer dengan instance dari dataset
        [PARAMS]
            input_instance      : list[int|float]   { sebuah input dari dataset }
        '''
        self._layers[0].nodes = input_instance
    
    def feed_forward(self, input_group, n_batch=None):
        if n_batch is None:
            n_batch = len(input_group)
        n_data = len(input_group)
        
        for i in range(math.ceil(n_data/n_batch)):
            print(f"{'#'*30}\nBatch {i+1}:")
            start_idx_data = i * n_batch
            end_idx_data = (i+1) * n_batch
            self.feed_forward_batch(input_group[start_idx_data:end_idx_data])
            
            # Back-propagation here
    
    def feed_forward_batch(self, input_group):
        '''
        [DESC]
            Melakukan proses feed forward untuk 1 batch sekaligus
        [PARAMS]
            input       : input dari file hmm atau dalam bentuk list atau Layer(?)
        [RETURN]
            output      : Layer     { layer output } \t(1)
            output      : list[int|float]     { list }     (2)
            # belum diputuskan yg mana 1/2
        '''
        hasil = [0] * self.n_input
        for i, masukan in enumerate(input_group):
            masukan_temp = [1] + masukan
            self.init_input_layer(masukan)
            print('Input Layer : [x1, x2] =', self._layers[0].nodes)
            for j in range(1, network_depth):
                layer = self._layers[j]
                w = np.array(self._weights[j-1])    # matrix of weights
                x = np.array(masukan_temp)          # array of input from lower layer
                sigma = np.matmul(w,x)              # matrix multiplication
                layer.nodes = sigma.tolist()        # assignment from result multiplication into array nodes
                text = 'Hidden Layer = [h1, h2] :' if (j < network_depth - 1) else 'Output Layer = y :'
                layer.activate()
                print(text, layer.nodes)
                    
                '''hasil aktivasi dijadikan sebagai input pada layer di atasnya
                perlu ditambahkan '1' sebagai bias untuk hidden layer'''
                if(j == network_depth-1):
                    masukan_temp = layer.nodes
                    break
                else:
                    masukan_temp = [1] + layer.nodes
                
            hasil[i] = masukan_temp
            print(f'Hasil data {i+1}: {masukan} -> {[round(e) for e in hasil[i]]} \n')

In [6]:
neural_network = NeuralNetwork(model)
neural_network.feed_forward(input_group)

##############################
Batch 1:
Input Layer : [x1, x2] = [0, 0]
Hidden Layer = [h1, h2] : [4.5397868702434395e-05, 0.9999999999999065]
Output Layer = y : [4.543910487654591e-05]
Hasil data 1: [0, 0] -> [0] 



## Visualization

In [7]:
from graphviz import Source

In [8]:
def generate_txt(neural_network: NeuralNetwork):
    '''
    [DESC]
        Buat visualisasi untuk model dalam format graphviz
    [PARAMS]
        model       : NeuralNetwork     { model yang akan divisualisasikan }
    '''
    # newline (\n) variabel, bcos backslashes (\) aren't allowed inside '''
    NL = f"{chr(10)}"
    layers = neural_network._layers
    n_layer = len(layers)
    input_layer = layers[0]
    hidden_layers = layers[1:-1]
    n_hidden_layers = len(hidden_layers)
    output_layer = layers[-1]

    graph_view_format = '''
        graph[ fontname = "Helvetica-Oblique",
            fontsize = 12,
            label = "",
            size = "7.75,10.25"
        ];
        rankdir = LR;
        splines=false;
        edge[style=invis];
        ranksep= 1.4;'''
    
    def generate_layer_node_names (sym, n, start=0):
        # Generate list of str containing node names in the
        # format of ["sym1", "sym2", "sym3", ..., "symn"]
        return [f"{sym}{i+start}" for i in range(n+1)]
    input_layer_node_names = generate_layer_node_names("x", input_layer._width)
    hidden_layers_node_names = [
        generate_layer_node_names(f"h{depth+1}", hidden_layers[depth]._width)
        for depth in range(n_hidden_layers)
    ]
    output_layer_node_names = generate_layer_node_names("y", output_layer._width-1, start=1)

    def layer_node_decl_template(node_name):
        # Generate node declaration string in format of
        # sym1 [label=<sym<sub>1</sub>>];
        node_sym = node_name[0]
        node_idx = node_name[1:]
        return f"{node_sym}{node_idx} [label=<{node_sym}<sub>{node_idx}</sub>>];"
    
    def layer_rank_template(node_names):
        # Generate concatenation string of
        # node names in the same layer
        return "->".join(node_names) + ";"
    
    # Input layer nodes declaration
    graph_input_layer_decl = f'''
        {{
            node [shape=circle, color=chartreuse, style=dotted, fillcolor=chartreuse];
            {layer_node_decl_template(input_layer_node_names[0])}
        }}
        {{
            node [shape=circle, color=chartreuse, style=filled, fillcolor=chartreuse];
            {NL.join(
                list(map(layer_node_decl_template, input_layer_node_names[1:]))
            )}
        }}
        {{
            rank=same;
            {layer_rank_template(input_layer_node_names)}
        }}{NL}'''

    # Hidden layer nodes declaration
    hidden_layer_rank_template = lambda : \
        NL.join(
            [f'''
            {{
            rank=same;
            {layer_rank_template(a_hidden_layer_node_names)}
            }}'''
            for a_hidden_layer_node_names in hidden_layers_node_names]
        )
    graph_hidden_layers_decl = f'''{NL.join(f"""{{
        node [shape=circle, color=dodgerblue, style=dotted, fillcolor=dodgerblue];
        {layer_node_decl_template(a_hidden_layer_node_names[0])}
    }}
    {{
        node [shape=circle, color=dodgerblue, style=filled, fillcolor=dodgerblue];
        {NL.join(
            list(map(layer_node_decl_template, a_hidden_layer_node_names))
            )}
    }}
    """ for a_hidden_layer_node_names in hidden_layers_node_names)}
    {hidden_layer_rank_template()}'''
    f'''
        {{
            node [shape=circle, color=dodgerblue, style=filled, fillcolor=dodgerblue];
            {NL.join(
                [NL.join(
                    list(map(layer_node_decl_template, a_hidden_layer_node_names))
                )
                for a_hidden_layer_node_names in hidden_layers_node_names]
            )}
        }}
        {hidden_layer_rank_template()}'''
    
    # Output layer nodes declaration
    graph_output_layer_decl = f'''
        {{
            node [shape=circle, color=coral1, style=filled, fillcolor=coral1];
        {NL.join(list(map(layer_node_decl_template, output_layer_node_names)))}
        }}
        {{
            rank=same;
            {layer_rank_template(output_layer_node_names)}
        }}'''

    # Layer names labelling
    def layer_labelling(type, activation, a_node_name, depth, start=0):
        return f'''
        l{depth} [shape=plaintext, label="Layer {depth+start} ({type} layer){NL}{activation}"];
        l{depth}->{a_node_name};
        {{rank=same; l{depth}; {a_node_name}}};
        '''
    graph_input_layer_labelling = layer_labelling("input", input_layer._type , input_layer_node_names[0], 0)
    graph_hidden_layers_labelling = ""
    for depth, hidden_layer_node_names in enumerate(hidden_layers_node_names):
        graph_hidden_layers_labelling += layer_labelling("hidden", hidden_layers[depth]._type, hidden_layer_node_names[0], depth+1)
    graph_output_layer_labelling = layer_labelling("output", output_layer._type, output_layer_node_names[0], n_layer-1)

    # Weights visualization

    graph_weights = ""
    all_node_names = [input_layer_node_names] + hidden_layers_node_names + [output_layer_node_names]
    for i in range(len(all_node_names[:-1])):
        line = f"{{{';'.join(all_node_names[i])}}} -> {{{';'.join(all_node_names[i+1])}}};\n"
        graph_weights += line
    
    graph_edges_txt = ""
    layers_node_names = [input_layer_node_names] + hidden_layers_node_names + [[""] + output_layer_node_names]
    for depth in range(n_layer-1):
        prev_layer_node_names = layers_node_names[depth]
        next_layer_node_names = layers_node_names[depth+1]
        weights = neural_network._weights[depth]
        for i, prev_node in enumerate(prev_layer_node_names):
            for j, next_node in enumerate(next_layer_node_names[1:]):
                weight = weights[j][i]
                if i == 0:
                    graph_edges_txt += f'{prev_node} -> {next_node} [xlabel="{weight}", style="dotted", arrowhead="open"]{NL}'
                else:
                    graph_edges_txt += f'{prev_node} -> {next_node} [xlabel="{weight}", style="filled", arrowhead="open"]{NL}'

    result_txt = "digraph G {" \
        + graph_view_format \
        + graph_input_layer_decl \
        + graph_hidden_layers_decl \
        + graph_output_layer_decl \
        + graph_input_layer_labelling \
        + graph_hidden_layers_labelling \
        + graph_output_layer_labelling \
        + "edge[style=solid, tailport=e, headport=w];" \
        + graph_edges_txt \
        + "}"

    return result_txt

def build_format(result_txt):
    from graphviz import Source
    dot = Source(result_txt)
    dot.format = 'png'
    dot.render('neural_network_01', view=False) 
    # 'neural_network_01.png'

result_txt = generate_txt(neural_network)
# print(result_txt)
build_format(result_txt)

IndexError: list index out of range

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c6d58e5-cc7c-4610-9ec1-8b211eea2409' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>